In [58]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from datetime import datetime
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [84]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
dataset=tf.keras.utils.get_file('aclImdb_v1.tar.gz',url,untar=True,cache_dir='.',cache_subdir='')
dataset_dir=os.path.join(os.path.dirname(dataset),'aclImdb')
dataset_dir

'./aclImdb'

In [61]:
os.listdir(dataset_dir)

['imdbEr.txt', 'train', 'test', 'README', 'imdb.vocab']

In [85]:
train_dir=os.path.join(dataset_dir,'train')
os.listdir(train_dir)

['pos',
 'unsupBow.feat',
 'labeledBow.feat',
 'neg',
 'urls_unsup.txt',
 'urls_neg.txt',
 'unsup',
 'urls_pos.txt']

In [86]:
remove_dir=os.path.join(train_dir,'unsup')
shutil.rmtree(remove_dir)

In [87]:
#taking reviews from the directory making bactches and spliting in 80:20
batch_size=1024
seed=123

train_ds=tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train',batch_size=batch_size,validation_split=0.2,subset='training',seed=seed)
val_ds=tf.keras.preprocessing.text_dataset_from_directory('aclImdb/train',batch_size=batch_size,validation_split=0.2,subset='validation',seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [88]:
#look at few samples
for text,label in train_ds.take(1):
  for i in range(0,5):
    print(label[i].numpy(),text[i].numpy())

0 b"Oh My God! Please, for the love of all that is holy, Do Not Watch This Movie! It it 82 minutes of my life I will never get back. Sure, I could have stopped watching half way through. But I thought it might get better. It Didn't. Anyone who actually enjoyed this movie is one seriously sick and twisted individual. No wonder us Australians/New Zealanders have a terrible reputation when it comes to making movies. Everything about this movie is horrible, from the acting to the editing. I don't even normally write reviews on here, but in this case I'll make an exception. I only wish someone had of warned me before I hired this catastrophe"
1 b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams of doing to his/her school. I absolutely loved the soundtrack as well as the carefully placed cynicism. If you like monty python, You will love this film. This movie is a tad bit "grease"esk (without

In [89]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [67]:
embedding_layer=tf.keras.layers.Embedding(1000,5)

In [68]:
result=embedding_layer(tf.constant([1,2,3]))
result.numpy()

array([[-3.1079268e-02, -2.7795469e-02, -3.5374902e-02, -7.5429566e-03,
         4.6634320e-02],
       [-3.5752904e-02, -9.8288059e-05,  2.9055897e-02, -4.1285027e-02,
         2.1931518e-02],
       [ 3.2352295e-02, -3.9433144e-02,  4.7483150e-02, -4.7392655e-02,
         2.8619058e-03]], dtype=float32)

In [69]:
#passing 2 sequence of length 3 we get 2,3,5
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
result.numpy()

array([[[-2.0337630e-02,  2.0280186e-02,  3.8174044e-02, -2.3523271e-02,
          4.6282325e-02],
        [-3.1079268e-02, -2.7795469e-02, -3.5374902e-02, -7.5429566e-03,
          4.6634320e-02],
        [-3.5752904e-02, -9.8288059e-05,  2.9055897e-02, -4.1285027e-02,
          2.1931518e-02]],

       [[ 3.2352295e-02, -3.9433144e-02,  4.7483150e-02, -4.7392655e-02,
          2.8619058e-03],
        [ 4.9909066e-02, -5.3691156e-03, -3.6219358e-03,  4.7625069e-02,
          6.9572553e-03],
        [ 1.7986666e-02,  4.9321186e-02, -1.9525027e-02, -4.3819238e-02,
         -6.9815628e-03]]], dtype=float32)

In [90]:
#create a custom

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')

vocab_size = 10000
sequence_length = 100

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [91]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

In [92]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [93]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [94]:
model.fit(train_ds,
    validation_data=val_ds, 
    epochs=15)

Epoch 1/15
20/20 [==============================] - 4s 184ms/step - loss: 0.6924 - accuracy: 0.5037 - val_loss: 0.6888 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 2s 96ms/step - loss: 0.6864 - accuracy: 0.5037 - val_loss: 0.6797 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 2s 90ms/step - loss: 0.6756 - accuracy: 0.5037 - val_loss: 0.6656 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 2s 90ms/step - loss: 0.6594 - accuracy: 0.5037 - val_loss: 0.6464 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 2s 94ms/step - loss: 0.6373 - accuracy: 0.5054 - val_loss: 0.6222 - val_accuracy: 0.5164
Epoch 6/15
20/20 [==============================] - 2s 99ms/step - loss: 0.6095 - accuracy: 0.5546 - val_loss: 0.5943 - val_accuracy: 0.5886
Epoch 7/15
20/20 [==============================] - 2s 94ms/step - loss: 0.5775 - accuracy: 0.6357 - val_loss: 0.5646 - val_accuracy: 0.6494
Epoch 8/15
2

In [95]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_9 (TextVe (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d_5 ( (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [96]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [98]:
weights

array([[ 0.0264771 , -0.01831573,  0.01649359, ...,  0.00405018,
        -0.03218202, -0.03627683],
       [ 0.14591056, -0.02213825,  0.08222561, ...,  0.06845948,
        -0.02321518, -0.09757018],
       [ 0.11963607, -0.0385525 ,  0.0644059 , ..., -0.0409118 ,
        -0.04076691, -0.2332143 ],
       ...,
       [ 0.14420667, -0.18309374,  0.16148348, ...,  0.14707728,
         0.23008019,  0.20004645],
       [-0.10745317,  0.18008916, -0.12716128, ..., -0.13768785,
        -0.10978431, -0.1686742 ],
       [ 0.00600218,  0.06314564, -0.05916734, ..., -0.06990606,
        -0.02860629, -0.09647774]], dtype=float32)

In [97]:
vocab

['',
 '[UNK]',
 'the',
 'and',
 'a',
 'of',
 'to',
 'is',
 'in',
 'it',
 'i',
 'this',
 'that',
 'was',
 'as',
 'with',
 'for',
 'movie',
 'but',
 'film',
 'on',
 'not',
 'you',
 'are',
 'his',
 'have',
 'be',
 'he',
 'one',
 'its',
 'at',
 'all',
 'by',
 'an',
 'they',
 'from',
 'who',
 'so',
 'like',
 'her',
 'just',
 'or',
 'about',
 'has',
 'out',
 'if',
 'some',
 'there',
 'what',
 'good',
 'more',
 'when',
 'very',
 'even',
 'she',
 'up',
 'my',
 'no',
 'would',
 'time',
 'only',
 'which',
 'really',
 'story',
 'their',
 'see',
 'were',
 'had',
 'can',
 'me',
 'we',
 'than',
 'much',
 'well',
 'been',
 'will',
 'get',
 'people',
 'bad',
 'also',
 'other',
 'do',
 'into',
 'great',
 'first',
 'because',
 'how',
 'most',
 'him',
 'dont',
 'made',
 'movies',
 'then',
 'them',
 'way',
 'films',
 'make',
 'could',
 'any',
 'after',
 'too',
 'characters',
 'think',
 'watch',
 'being',
 'two',
 'many',
 'seen',
 'character',
 'plot',
 'little',
 'never',
 'acting',
 'where',
 'best',
 '

In [104]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  print(word)
  if index==10:
    break
  # if  index == 0: continue # skip 0, it's padding.
  # vec = weights[index] 
  # out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  # out_m.write(word + "\n")
out_v.close()
out_m.close()


[UNK]
the
and
a
of
to
is
in
it
i


In [100]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception as e:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>